In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

from IPython.display import clear_output
!pip -q install dicomsdl
!pip install pylibjpeg
!pip install python-gdcm
!pip install plotly==5.11.0
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: plotly
    Found existing installation: plotly 5.14.1
    Uninstalling plotly-5.14.1:
      Successfully uninstalled plotly-5.14.1


In [2]:
import os, random, cv2, dicomsdl
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

from tqdm import tqdm
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from mpl_toolkits.axes_grid1 import ImageGrid
import pydicom
import pylibjpeg
from pathlib import Path
sns.set_style("darkgrid")

/opt/conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/opt/conda/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
/tmp/ipykernel_32/878433829.py:4: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling`

In [3]:
DEBUG=False

In [4]:
path = '/kaggle/input/rsna-breast-cancer-detection/'
test_csv = path + 'test.csv'
test_path = path + 'test_images/'
sample_submission = path + 'sample_submission.csv'
output_path = '/kaggle/working/'
test_output_path = output_path + 'test_images/'

img_size =1024
resize_dim = (1024,512)

In [5]:
test_df = pd.read_csv(test_csv)
test_df.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [17]:
test_df = pd.read_csv(sample_submission)
test_df.head()

,prediction_id,cancer
0,10008_L,0.021168
1,10008_R,0.021168


In [6]:
def dicom_to_png(dicom_path):
    dicom = dicomsdl.open(dicom_path)
    image = dicom.pixelData(storedvalue=False)
    image = image - np.min(image)
    image = image / np.max(image)

    if dicom.PhotometricInterpretation == 'MONOCHROME1':
        image = 1.0 - image
    
    image = cv2.resize(image, (img_size, img_size), interpolation=cv2.INTER_LINEAR)
    image = (image * 255).astype(np.uint8)
    return image

def png_to_roi(image, image_path):
    bin_image = cv2.threshold(image, 20, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(bin_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)
    ys = contour.squeeze()[:, 0]
    xs = contour.squeeze()[:, 1]
    roi = image[np.min(xs):np.max(xs), np.min(ys):np.max(ys)]
    return cv2.resize(roi, resize_dim[::-1], interpolation=cv2.INTER_LINEAR)

def process(dicom_path, image_path):
    image = dicom_to_png(dicom_path)
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    image = png_to_roi(image, image_path)
    cv2.imwrite(image_path, image)

In [7]:
test_df['dicom_path'] = test_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.dcm'
test_df['image_path'] = test_output_path + test_df['patient_id'].astype(str) + '/' + test_df['image_id'].astype(str) + '.png'
test_df.head()

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id,dicom_path,image_path
0,2,10008,736471439,L,MLO,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/736471439.png
1,2,10008,1591370361,L,CC,81,0,21,10008_L,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/1591370361.png
2,2,10008,68070693,R,MLO,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/68070693.png
3,2,10008,361203119,R,CC,81,0,21,10008_R,/kaggle/input/rsna-breast-cancer-detection/tes...,/kaggle/working/test_images/10008/361203119.png


In [8]:
Parallel(n_jobs=4, backend='threading')(delayed(process)(dicom_path, image_path) 
                   for dicom_path, image_path in tqdm(zip(test_df['dicom_path'], 
                                                          test_df['image_path'])))
clear_output()

In [9]:
!cp /kaggle/input/rsna-breast-cancer-detection/train.csv /kaggle/working/
!cp /kaggle/input/rsna-breast-cancer-detection/test.csv /kaggle/working/
!cp /kaggle/input/rsna-breast-cancer-detection/sample_submission.csv /kaggle/working/

In [10]:
from tensorflow.keras.models import load_model

def predict_model(model_name, generator):
    model = load_model(model_name)
    pred = model.predict(generator)
    
    y_pred = []
    for prob in pred:
        if prob >= 0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    
    y_true = generator.classes
    
    return (y_pred, y_true, pred)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [11]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    #dir_path_test,
    "/kaggle/working/test_images/",
    target_size = (1024, 1024),
    batch_size = 128,
    class_mode = 'binary')

Found 4 images belonging to 1 classes.


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight

In [13]:
y_pred, y_true, test_pred = predict_model('/kaggle/input/baseline/baseline_vgg16.h5', test_generator)

if DEBUG:
    cm = confusion_matrix(y_true, y_pred)
    cm_pct = confusion_matrix(y_true, y_pred, normalize = "true")

    class_names = ['Negativo', 'Positivo']

    ax = sns.heatmap(cm_pct*100, annot = True, fmt = '.2f', cmap = "Blues", annot_kws = {"size": 16}, xticklabels = class_names, yticklabels = class_names)

    i=0
    for t in ax.texts:
      t.set_text(str(cm.flatten()[i])+ "\n("+ t.get_text() + "%)")
      i = i+1

    ax.set_xlabel("Predição")
    ax.set_ylabel("Real")

1/1 [==============================] - 25s 25s/step


In [14]:
def pfbeta(labels, predictions, beta):
    y_true_count = 0
    ctp = 0
    cfp = 0

    for idx in range(len(labels)):
        prediction = min(max(predictions[idx], 0), 1)
        if (labels[idx]):
            y_true_count += 1
            ctp += prediction
        else:
            cfp += prediction

    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp)
    c_recall = ctp / y_true_count
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return result
    else:
        return 0

In [15]:
pf = pfbeta(test_generator, test_pred, 1)
pf

array([0.57931346], dtype=float32)

In [16]:
test_df["cancer"] = (test_pred > 0.5).astype(int)
result = test_df[['prediction_id', 'cancer']].groupby("prediction_id").mean().reset_index()
result.to_csv('/kaggle/working/submission.csv', index=False)